In [5]:
import cv2
import sys, os
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import imageio
import numpy as np
from numpy import asarray

# DeepFake Detection Algorithm
Input : $D_{real}$, $D_{fake}$ were centered by subtracting the mean of the real training data.

## Create $D_{real}$

In [6]:
# Read images
img_real = []
path = '/content/drive/MyDrive/American_University/2021_Fall/DATA-793-001_Data Science Practicum/Datasets/manipulated_sequences/Deepfakes/raw/videos/data_test'

for root, _, files in os.walk(path):
    current_directory_path = os.path.abspath(root)
    for f in files:
        name, ext = os.path.splitext(f)
        if ext == ".jpg":
            current_image_path = os.path.join(current_directory_path, f)
            current_image = imageio.imread(current_image_path)
            img_real.append(current_image)
img_real = np.array(img_real, dtype=object)
for img in img_real:
  print(img.shape)
print(img_real.shape)

(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(480, 854, 3)
(480, 854, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(10,)


## Global Centering for each image

In [4]:
def centering(arr):
  new_arr = []
  for i in range(len(arr)):
    pixels = arr[i]
    #pixels = pixels.astype('float32') # convert from integers to floats
    mean = pixels.mean() # calculate global mean
    pixels = pixels - mean # centering of pixels
    new_arr.append(pixels)
  new_arr = np.array(new_arr, dtype=object)
  return new_arr

img_real = centering(img_real)
print(len(img_real))

10


# Preprocessing and data tensor organization

In [7]:
print(img_real.shape)
[U_real, S_real, V_real] = np.linalg.svd(img_real[0])

(10,)


In [12]:
for i in range(len(img_real)):
  [U_real, S_real, V_real] = np.linalg.svd(img_real[i])
  

(720, 3, 3)


In [ ]:
img_df = []           
for img in images:
  img = img.flatten()
  img_df.append(img)
  print(img.shape)
img_df = np.array(img_df, dtype=object)
print(img_df)